# Download representative images of volcanos


Credit: this script derived from [Aaron Geller's blog post.](https://sites.northwestern.edu/researchcomputing/2021/11/19/downloading-satellite-images-made-easy/), including the use of functions based on his [EarthEngineToGeoTIFF repository.](https://github.com/ageller/EarthEngineToGeoTIFF)

In [30]:
import ee
%pip install rasterio

import rasterio
from rasterio.plot import show as showRasterio

import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import date
from datetime import timedelta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
# connect to google drive
from google.colab import drive  

# drive roots
DRIVE_MOUNT = '/content/drive'
drive.mount(DRIVE_MOUNT, force_remount=True)

# set up project directories
PROJECT_DIR = DRIVE_MOUNT + "/My Drive/Volcano_datasets/"
TMP_DIR = PROJECT_DIR + "TempDIR/"
OUTPUT_IMAGE_DIR = PROJECT_DIR +"Satellite_images/"



targets = pd.read_excel(PROJECT_DIR + "Sample Eruptions.xlsx")



Mounted at /content/drive


In [51]:
import zipfile
import os
import requests

import numpy as np

# percent cloud filter allowable
CLOUD_FILTER = 50

def getSentinalS2SRImage(lon, lat, sze_km, rgb_filename, nir_filename,
                         dateMin = '2020-04-01', dateMax = '2020-04-30', 
                         min = 0, vmax = 3500,
                         temp_dir = TMP_DIR):
    '''    
    download an RGB image from the Sentinal S2 Surface Reflectance satellite, at the given coordinates
    
    lon : central longitude in degrees
    lat : central latitude in degrees
    sze_km : size of the edge of the box in km
    dateMin : minimum date to use for image search in year-month-day (e.g., 2020-08-01)
    dateMax : maximum date to use for image search in year-month-day (e.g., 2020-08-31)
    vMin : minimum value to select in the Sentinal image pixels (I think this should be close to 0)
    vMax : maximum value to select in the Sentinal image pixels (I think this should be close to 3000)
    rgb_filename : output filename for the RGB GeoTIFF image
    nir_filename : output filename for the RGB GeoTIFF image
    
    Note: it's possible that the vMin and vMax values should be different for each band to make the image look nicer
    
    https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR
    '''


    print('Downloading Sentinel S2 Surface Reflectance satellite images ... ')
    
    # define the area of interest, using the Earth Engines geometry object

    pt = ee.Geometry.Point(lon,lat)
    
    aoi = ee.Geometry.buffer(pt,sze_km)

    # get the image using Google's Earth Engine
    db = ee.Image(ee.ImageCollection('COPERNICUS/S2_SR')\
                       .filterBounds(aoi)\
                       .filterDate(ee.Date(dateMin), ee.Date(dateMax))\
                       .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))\
                       .mosaic())
    
    # add the latitude and longitude
    db = db.addBands(ee.Image.pixelLonLat())


    # define the bands that I want to use.  B4 is red, B3 is green, B2 is blue, b8 is near IR, 
    # https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR#bands
    bands = ['B4', 'B3', 'B2','B8']

    # export geotiff images, these go to Drive and then are downloaded locally
    for selection in bands:
        task = ee.batch.Export.image.toDrive(image=db.select(selection),
                                     description=selection,
                                     scale=30,
                                     region=aoi,
                                     fileNamePrefix=selection,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')
        task.start()

        url = db.select(selection).getDownloadURL({
            'scale': 30,
            'crs': 'EPSG:4326',
            'fileFormat': 'GeoTIFF',
            'region': aoi})
    
        r = requests.get(url, stream=True)

        filenameZip = temp_dir + selection+'.zip'
        filenameTif = selection+'.tif'

        # unzip and write the tif file, then remove the original zip file
        with open(filenameZip, "wb") as fd:
            for chunk in r.iter_content(chunk_size=1024):
                fd.write(chunk)

        zipdata = zipfile.ZipFile(filenameZip)
        zipinfos = zipdata.infolist()

        # iterate through each file (there should be only one)
        for zipinfo in zipinfos:
            zipinfo.filename = filenameTif
            zipdata.extract(zipinfo,path=temp_dir)
    
        zipdata.close()
        
    # create a combined RGB geotiff image
    # https://gis.stackexchange.com/questions/341809/merging-sentinel-2-rgb-bands-with-rasterio
    print('Creating 3-band GeoTIFF image ... ')
    
    # open the images
    B2 = rasterio.open(temp_dir +'B2.tif')
    B3 = rasterio.open(temp_dir +'B3.tif')
    B4 = rasterio.open(temp_dir +'B4.tif')
    B8 = rasterio.open(temp_dir +'B8.tif')

    # get the scaling
    image = np.array([B2.read(1), B3.read(1), B4.read(1)]).transpose(1,2,0)
    p2, p98 = np.percentile(image, (2,98))

    # use the B3 image as a starting point so that I keep the same parameters
    B3_geo = B3.profile
    B3_geo.update({'count': 3})

    with rasterio.open(rgb_filename, 'w', **B3_geo) as dest:
        dest.write( (np.clip(B4.read(1), p2, p98) - p2)/(p98 - p2)*255, 1)
        dest.write( (np.clip(B3.read(1), p2, p98) - p2)/(p98 - p2)*255, 2)
        dest.write( (np.clip(B2.read(1), p2, p98) - p2)/(p98 - p2)*255, 3)

    dest.close()

    # Near IR replaces the blue channel
    with rasterio.open(nir_filename, 'w', **B3_geo) as dest:
        dest.write( (np.clip(B4.read(1), p2, p98) - p2)/(p98 - p2)*255, 1)
        dest.write( (np.clip(B3.read(1), p2, p98) - p2)/(p98 - p2)*255, 2)
        dest.write( (np.clip(B8.read(1), p2, p98) - p2)/(p98 - p2)*255, 3)

    B2.close()
    B3.close()
    B4.close()
    B8.close()
    
    # remove the intermediate files
    for selection in bands:
        os.remove(temp_dir +selection + '.tif')
        os.remove(temp_dir +selection + '.zip')
  


In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=jI7CtqZZNcNrvgGMHQr-aImA_tzNZI1cFu-rMwGZliE&tc=5eYR_B5yW3GwgBtdiF1JqAdT5IcTPFbjNzWwNV_Zldw&cc=VvFqmIVMUpC7CMCvZY3qch9o-RDUG4LRFBgGvbQX71s

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWj1xvNGWuhQ1NphU6cBWS4qbxzlol14MVTWi2JRxFkOWxqSw3VjXVE

Successfully saved authorization token.


In [48]:
def get_image_data(longitude, latitude, buffer_size,
                             output_rgb,output_grn, 
                            dateMin , dateMax ):
   # download stacked tif files
    _ = getSentinalS2SRImage(longitude, latitude, buffer_size,
                             output_rgb+".tif",output_grn+".tif", 
                            dateMin , dateMax )

    #convert from tif to png
    for file in [output_rgb,output_grn]:
        f,ax = plt.subplots(figsize=(15,15))

        img = rasterio.open(file+".tif")

        showRasterio(img.read(), ax = ax, transform=img.transform)

        f.savefig(file+".png", bbox_inches='tight')

        img.close()

        os.remove(file + ".tif")

    


In [ ]:
target_size_km = 50000
for i_eruption in range(8,targets.shape[0]):
    volc = targets.loc[i_eruption]
    v_name = re.sub(r"[,.]","",re.sub(r"\s+", "_",volc["Volcano Name"]))
    print(v_name)
    erupt_start_date = date(volc["Start Year"],volc["Start Month"],volc["Start Day"])
    erupt_end_date = date(volc["End Year"],volc["End Month"],volc["End Day"])
    next_start_date = erupt_start_date
    next_end_date = erupt_start_date + timedelta(days=7)
    while (next_end_date <= erupt_start_date + timedelta(days=7)):
        volc_file_name = f'{OUTPUT_IMAGE_DIR}{volc["Volcano Number"]:.0f}_{v_name}_{volc["Eruption Number"]:.0f}_{next_start_date}'

        print(i_eruption, volc_file_name)
       
        get_image_data(volc["Longitude"], volc["Latitude"], target_size_km,
                             volc_file_name,volc_file_name+"_GRN", 
                            dateMin = next_start_date.isoformat(), dateMax=next_end_date.isoformat())
        next_start_date = next_start_date + timedelta(days=7)
        next_end_date = next_end_date + timedelta(days=7)
       

Kavachi
8 /content/drive/My Drive/Volcano_datasets/Satellite_images/255060_Kavachi_22364_2020-09-02
